# __Follwing Social Network Creation__

In [1]:
import pandas as pd
import datetime, os, tweepy
from credentials import *
import time, tweepy
from IPython.display import clear_output, display
import networkx as nx

### Getting the nodes (users) from all days

In [2]:
complete_network = nx.read_gexf('data/networks/complete_network.gexf')

In [3]:
len(list(complete_network.nodes()))

325154

In [4]:
complete_network = nx.read_gexf('data/networks/complete_network.gexf')
user = []

for u, d in list(complete_network.nodes(data=True)):
    user.append((u, d['tt_id']))

In [5]:
user[:10]

[('seguezaqueu', 1247224168850034694),
 ('zaqueusdvPT', 1573595167),
 ('OSul_noticias', 3245657679),
 ('farah_julioo', 1164340987071750145),
 ('thetunnes', 1360024018246631425),
 ('joaodoria45', 3405608584),
 ('eriicksonsilva', 51587901),
 ('Elmosol', 82350206),
 ('silviacbap', 736942609663467520),
 ('DarlyMenconi', 1243642241186516992)]

In [6]:
usrs_df = pd.DataFrame(user, columns=['displayname', 'id'])
usrs_df.head()

,displayname,id
0,seguezaqueu,1247224168850034694
1,zaqueusdvPT,1573595167
2,OSul_noticias,3245657679
3,farah_julioo,1164340987071750145
4,thetunnes,1360024018246631425


In [7]:
usrs_df['id'].value_counts()

1365751729254465546    7
1219062699843301376    4
1329969912413040647    4
1427728286621253643    4
715226642289963008     4
                      ..
98503559               1
1191488295701766149    1
199022678              1
832060527442538498     1
1341947278165204992    1
Name: id, Length: 324333, dtype: int64

In [8]:
usrs_df.drop_duplicates(subset='id', keep='last', inplace=True, ignore_index=True)
usrs_df.shape

(324333, 2)

In [9]:
usrs_df['id'].value_counts()

1247224168850034694    1
2361832971             1
1291916216685268993    1
1376322458458918913    1
1145217039369457665    1
                      ..
1048978269012484097    1
1257087895858556928    1
1238251109518577664    1
1389660295392600071    1
1341947278165204992    1
Name: id, Length: 324333, dtype: int64

In [10]:
usrs_df.to_parquet('data/networks/node_id.parquet')

In [11]:
del complete_network, user

In [12]:
#nx.read_gexf('data/networks/complete_network.gexf').nodes(data=True)

In [13]:
#network = nx.read_gexf('data/networks/complete_network.gexf')

In [14]:
#for u,d in list(network.nodes(data=True)):
#    if d['label']:
#        print(u)

In [15]:
#len(users)

In [16]:
#users[:15]

In [17]:
#DATA_PATH_TT = 'data/tweets/'
#DATA_PATH_NT = 'data/networks/'

In [18]:
#week_list = [week_dir for week_dir in os.listdir(DATA_PATH_NT) if os.path.isdir(DATA_PATH_TT+week_dir) and not week_dir.endswith('.ipynb_checkpoints')]

#week_list.sort()
#week_list.remove('week_pr_01')
#week_list.insert(week_list.index('week_12')+1, 'week_pr_01')
#week_list.remove('week_pr_02')
#week_list.insert(week_list.index('week_pr_01')+1, 'week_pr_02')
#week_list.remove('week_pr_03')
#week_list.insert(week_list.index('week_17')+1, 'week_pr_03')
#week_list.remove('week_pr_04')
#week_list.insert(week_list.index('week_21')+1, 'week_pr_04')

#print(week_list)

In [19]:
#day_list = ['day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7']
#node_list = []

#for week in week_list:
#    print(f'Starting week {week}...\n')
#    for day in day_list:
#        print(day+'...')
#        nodes = pd.read_csv(DATA_PATH_NT+week+'/'+day+'/'+'gephi_nodes.csv')
#        for usr, disp_n, desc, tt_id in nodes[['Id', 'DisplayName', 'Description', 'TtId']].values:
#            if (usr, disp_n, desc, tt_id) not in node_list:
#                node_list.append((usr, disp_n, desc, tt_id))

#all_nodes = pd.DataFrame(node_list, columns=['Id', 'DisplayName', 'Description', 'TtId'])
#all_nodes.drop_duplicates(subset=['Id'], keep='last', ignore_index=True, inplace=True)
#all_nodes.to_csv(DATA_PATH_NT+'all_nodes.csv', index=False)

In [20]:
#all_nodes.drop_duplicates(subset='Id', keep='last', inplace=True, ignore_index=True)
#all_nodes.shape

In [21]:
#all_nodes.to_csv(DATA_PATH_NT+'all_nodes.csv', index=False)

### Following Social Network Edges

In [22]:
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [25]:
follows_list = []
susp_account = []
protec_usr = []
count = 0
n_usr = usrs_df.shape[0]

for usr_n, usr_id in usrs_df.values:
    clear_output(wait=True)
    print(f'''
    User number {count}: {usr_n}
    {n_usr - count} to go
    {(count/n_usr)*100}% finished
    Number of following links: {len(follows_list)}
    ''')
    try:
        if api.get_user(screen_name=usr_n).protected:
            protec_usr.append(usr_n)
            time.sleep(5)
        else:
            usr_follow = api.friends_ids(usr_n)
            for id_ in usr_follow:
                if id_ in usrs_df['id']:
                    follows_list.append((usr_id, id_))
            #for usr2 in all_nodes['Id']:
            #    if usr != usr2:
            #        friendship = api.get_friendship(source_screen_name=usr, target_screen_name=usr2)
            #        if friendship[0].following:
            #            follows_list.append((usr, usr2))
    #except tweepy.errors.TweepyException as e:
            time.sleep(5)
    except tweepy.TweepError as e:
        susp_account.append(usr_n)
        time.sleep(5)
    count+=1


    User number 15234: martafmartins
    309099 to go
    4.697024354598515% finished
    Number of following links: 2521
    


KeyboardInterrupt: 

In [26]:
follows_list[:10]

[(220450289, 30973),
 (34996429, 6503),
 (34996429, 4124),
 (717037526, 79543),
 (62587797, 572),
 (62587797, 33423),
 (17520030, 4816),
 (17520030, 3144),
 (1281766129, 20),
 (1281766129, 12)]

In [28]:
follow_df = pd.DataFrame(follows_list, columns=['Source', 'Target'])

In [31]:
follow_df.to_parquet('data/networks/following_network.parquet', index=False)

In [ ]:
#friends = api.get_friends(screen_name='lucasRniere')

In [ ]:
#for user in friends:
#    print(user._json['screen_name'])

In [ ]:
#api.get_friendship(source_screen_name='lucasRniere', target_screen_name='Fazendo_Nerdice')
##api.exists_friendship(source_screen_name='lucasRniere', target_screen_name='Fazendo_Nerdice')

In [ ]:
#api.friends_ids('lucasRniere')

In [ ]:
#for x in range(0,1000):
#    q = x/1000
#    p = q*100
#    print(f'Number: {x} - {p:.2f}%')
#    time.sleep(0.5)
#    clear_output(wait=True)
    

In [ ]:
api.get_user('lucasrniere').protected